# Intro to AI - Coursework
Joshua Luke Boddy - aczc760

Saeed Almansoori - 

### To-Do
- Functions to prepare each column of the dataset we need
- Decide on learning model
- Research integer value return

## Data Preparation

Taking a CSV file as our data set, the data needs to be converted across from CSV to something program readable as well as formatting the data into a readable format for our model (undecided).

In [ ]:
import pandas as pd

In [ ]:
def prepare_categorical_column(value, key):
    # Convert Categorical Data in Column to Binary
    # - Add new key to conversion_keys
    # - Check keys inside of key for value
    # - If value exists, rewrite table with new value
    # - Else, add value and assign new value
    # - Rewrite table with new value

In [10]:
cars_data = pd.read_csv('./cars.csv')
conversion_keys = {}

In [ ]:
def prepare_dataset():
    # Iterate over the columns inside the DataFrame
    # For each column, run prepare_categorical_column as a lambda